# Fall Detection using SisFall Dataset
Daniela Dias, nMec 98039

In [0]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, \
    roc_curve
import matplotlib.pyplot as plt

In [ ]:
# Load the preprocessed dataset from CSV files
X_train = pd.read_csv('deep_learning/X_train.csv')
X_test = pd.read_csv('deep_learning/X_test.csv')
y_train = pd.read_csv('deep_learning/y_train.csv')
y_test = pd.read_csv('deep_learning/y_test.csv')

## Utility Functions

In [ ]:
def confusion_matrix_plot(y_test, y_pred, title):
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Plot the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Fall", "No Fall"])
    disp.plot(cmap='Blues')
    disp.ax_.set_title(title)
    disp.ax_.set_xlabel("Predicted")
    disp.ax_.set_ylabel("True")
    disp.figure_.set_size_inches(8, 6)
    disp.figure_.tight_layout()
    plt.show()

## Convolutional Neural Network (CNN)

In [ ]:
# Reshape the data to fit the CNN input shape
X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1], 1)

In [ ]:
# Convert the labels to categorical format
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

In [ ]:
# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1, X_train.shape[2], 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(y_train.shape[1], activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred)
print('ROC AUC score:', roc_auc)

In [ ]:
# Calculate F1 score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print('F1 score:', f1)

In [ ]:
# Print classification report
print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
# Plot confusion matrix
confusion_matrix_plot(y_true_classes, y_pred_classes, "CNN Confusion Matrix")

In [ ]:
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true_classes, y_pred[:, 1])
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()